In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from complexLayers import ComplexBatchNorm2d, ComplexConv2d, ComplexLinear
from complexLayers import ComplexDropout2d, NaiveComplexBatchNorm2d
from complexFunctions import complex_relu, complex_max_pool2d

In [2]:
batch_size = 64
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = datasets.MNIST('../data', train=True, transform=trans, download=True)
test_set = datasets.MNIST('../data', train=False, transform=trans, download=True)

train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= batch_size, shuffle=True)

0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...


/opt/miniconda/py3/envs/py38/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [5]:
class ComplexNet(nn.Module):
    
    def __init__(self):
        super(ComplexNet, self).__init__()
        self.conv1 = ComplexConv2d(1, 10, 5, 1)
        self.bn  = ComplexBatchNorm2d(10)
        self.conv2 = ComplexConv2d(10, 20, 5, 1)
        self.fc1 = ComplexLinear(4*4*20, 500)
        self.dropout = ComplexDropout2d(p = 0.3)
        self.fc2 = ComplexLinear(500, 10)
             
    def forward(self,x):
        print(f"1 : {x.shape}")
        x = self.conv1(x)
        print(f"2 : {x.shape}")
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        print(f"3 : {x.shape}")
        x = self.bn(x)
        print(f"4 : {x.shape}")
        x = self.conv2(x)
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = x.view(-1,4*4*20)
        x = self.fc1(x)
        x = self.dropout(x)
        x = complex_relu(x)
        x = self.fc2(x)
        x = x.abs()
        x =  F.log_softmax(x, dim=1)
        return x
    
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ComplexNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target =data.to(device).type(torch.complex64), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {:3} [{:6}/{:6} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.item())
            )

In [6]:
# Run training on 4 epochs
for epoch in range(4):
    train(model, device, train_loader, optimizer, epoch)

1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
Train Epoch:   0 [     0/ 60000 (  0%)]	Loss: 2.595914
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 2

4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64

4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64

1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64, 10, 12, 12])
4 : torch.Size([64, 10, 12, 12])
1 : torch.Size([64, 1, 28, 28])
2 : torch.Size([64, 10, 24, 24])
3 : torch.Size([64

KeyboardInterrupt: 